In [1]:
%matplotlib inline
from copy import deepcopy
from os.path import abspath
from typing import Dict, Any
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve, GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import xgboost
import lightgbm

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
pd.options.display.max_columns = None

In [ ]:
source = abspath("./data.csv")

In [ ]:
df = pd.read_csv(source, sep=";")

# Вернём значения
Напишем мапперы значений, которые авторы датасета закодировали. Все числовые отрезки заменим на число из этого отрезка, чаще всего середину.

In [ ]:
decode_income_map = {
    1: 7500.,
    2: 12500.,
    3: 17500.,
    4: 22500.,
    5: 27500.,
    6: 35000.,
    7: 45000.,
    8: 60000.,
    9: 90000.
}

In [ ]:
decode_sex_map = {
    1: "m",
    2: "f"
}

In [ ]:
decode_marital_status_map = {
    1: "Married",
    2: "Living together, not married",
    3: "Divorced or separated",
    4: "Widowed",
    5: "Single, never married"
}

In [ ]:
age_map = {
    1: 16,
    2: 21,
    3: 30,
    4: 40,
    5: 50,
    6: 60,
    7: 75
}

In [ ]:
education_map = {
    1: "Grade 8 or less",
    2: "Grades 9 to 11",
    3: "Graduated high school",
    4: "1 to 3 years of college",
    5: "College graduate",
    6: "Grad Study",
}

In [ ]:
occupation_map = {
    1: "Professional/Managerial",
    2: "Sales Worker",
    3: "Factory Worker/Laborer/Driver",
    4: "Clerical/Service Worker",
    5: "Homemaker",
    6: "Student, HS or College",
    7: "Military",
    8: "Retired",
    9: "Unemployed"
}

In [ ]:
living_period_map = {
    1: 0,
    2: 2,
    3: 5,
    4: 9,
    5: 15
}

In [ ]:
dual_incomes_map = {
    1: "Not Married",
    2: "Yes",
    3: "No"
}

In [ ]:
householder_status_map = {
    1: "Own",
    2: "Rent",
    3: "Live with Parents/Family",
}

In [ ]:
home_type_map = {
    1: "House",
    2: "Condominium",
    3: "Apartment",
    4: "Mobile Home",
    5: "Other",
}

In [ ]:
ethnic_map = {
    1: "American Indian",
    2: "Asian",
    3: "Black",
    4: "East Indian",
    5: "Hispanic",
    6: "Pacific Islander",
    7: "White",
    8: "Other",
}

In [ ]:
language_map = {
    1: "English",
    2: "Spanish",
    3: "Other",
}

In [ ]:
def decode_column(x: Any, mapp: Dict[Any, Any]) -> Any:
    if pd.isna(x): 
        return x
    return mapp[x]

In [ ]:
maps_for_visualisation = {
    "INCOME": decode_income_map,
    "SEX": decode_sex_map,
    "MARITAL_STATUS": decode_marital_status_map,
    "AGE": age_map,
    "EDUCATION": education_map,
    "OCCUPATION": occupation_map,
    "LIVING_PERIOD": living_period_map,
    "DUAL_INCOMES": dual_incomes_map,
    "HOUSEHOLDER_STATUS": householder_status_map,
    "HOME_TYPE": home_type_map,
    "ETHNIC": ethnic_map,
    "LANGUAGE": language_map,
}

maps_for_models = {
    "INCOME": decode_income_map,
    "SEX": decode_sex_map,
    "MARITAL_STATUS": decode_marital_status_map,
    "AGE": age_map,
    "OCCUPATION": occupation_map,
    "LIVING_PERIOD": living_period_map,
    "DUAL_INCOMES": dual_incomes_map,
    "HOUSEHOLDER_STATUS": householder_status_map,
    "HOME_TYPE": home_type_map,
    "ETHNIC": ethnic_map,
    "LANGUAGE": language_map,
}


In [ ]:
# for column in df.columns:
#     if column in maps_for_visualisation:
#         df[column] = df[column].apply(decode_column, args = (maps_for_visualisation,))  

In [ ]:
df.sample(20)

In [ ]:
def applay_maps(df: pd.DataFrame, maps: Dict[str, Dict[Any, Any]]) -> pd.DataFrame:
    df = df.copy()
    for column in df.columns:
        if column in maps:
            df[column] = df[column].apply(decode_column, args = (maps[column],))  
    return df

In [ ]:
df2 = df.copy()
df2 = applay_maps(df, maps_for_models)
df2.sample(10)

# Посмотрим на распределение параметров

In [ ]:
def show_statistic(df: pd.DataFrame) -> None:
    for column in df.columns:
        print(column)
        counts = df[column].value_counts().sort_index()
        
        names = counts.index
        if column in maps_for_visualisation:
            names = [str(decode_column(i, maps_for_visualisation[column])) for i in names]
            
        counts = counts.values
        plt.bar(names, counts)
        plt.xticks(rotation=45, ha="right")
        plt.show()
show_statistic(df)

    1) Имеем достаточно много одиноких покупателей. Можно предлагать товары, которые упрощают ведение хозяйства.
    2) Получаем, что женщины чаще ходят в наш супермаркет. Может оказаться так, что в семьях чаще покупками занимаются женщины. Стоит посмотреть соотношение одиноких женщин и одиноких мужчин, если и там баланс будет нарушен, значит есть потенциал развития магазина для одного из полов.
    3) Большинство клиентов прожили достаточно долго на одном месте, стоит проверить, являются ли они постоянными покупателями. Если нет - пересмотреть программу лояльности.
    4) Достаточно большое кол-во людей живёт с родителями. Если они достаточно взрослые, то можно предлагать им товары для пожилых.
    5) Много клиентов живут в домах, стоит проверить достаточно ли товаров для придомовой территории: газонокосилки, поливалки, уличная мебель, грили и т.д.
    6) Достаточно много "Испанцев", можно сделать предположение, что на самом деле это Мексиканцы. Можно сделать акцент на национальной кухне.

In [ ]:
single_df = df2[(df2["MARITAL_STATUS"] != "Married") & (df2["MARITAL_STATUS"] != "Living together, not married")]
counts = single_df["SEX"].value_counts().sort_index()
print(counts["m"] / counts["f"])
counts = df2["SEX"].value_counts().sort_index()
print(counts["m"] / counts["f"])

Видим, что соотношение одиноких мужчин/женщин ещё ниже. Если посмотреть соотношение полов в США в 1990 году, то увидим, что соотношение составляет ~0,95. https://www.statista.com/statistics/241495/us-population-by-sex/
Наши значения говорят о том, что есть не раскрытый потенциал для одиноких мужчин. Конечно, многое зависит от способа проведения запроса, что может влиять на выводы. Например, если опрос проводился только один день, а одинокие мужчины закупаются реже.

Посмотрим ещё распределение по доходу на одного человека.

In [ ]:
income_per_person = df2["INCOME"] / df2["PERSONS_COUNT"]
counts = income_per_person.value_counts(bins=20).sort_index()
indeces = [str(i.left) + "-" + str(i.right) for i in counts.index]
plt.figure(figsize=(15, 15))
plt.bar([str(int(i.left)) + "-" + str(int(i.right)) for i in counts.index], counts.values)
plt.xticks(rotation=45, ha="right")
plt.show()


Существует группа клиентов, получающих доход на одного человека, который выше, чем средний доход на одного человека в Сан-Франциско в 1987 (~$23700) https://fred.stlouisfed.org/series/PCPI06075 . Стоит рассмотреть их продуктовую корзину. Можно увеличить кол-во более дорогих товаров лучшего качества, если их нет в этой корзине.
При построении модели можно сгенерить фичу на основе кол-ва человек и значении среднего дохода на человека.

In [ ]:
corr = df.corr(method="kendall")
corr.style.background_gradient(cmap='coolwarm')

Посмотрим на зависимости признаков, особенно на те, которые коррелируют, чтобы понять какие именно есть зависимости.

In [ ]:
sns.catplot(x="AGE", hue="INCOME", height=8.27, aspect=11.7/8.27, kind="count", data=df2);

In [ ]:
sns.catplot(x="EDUCATION", hue="INCOME", height=8.27, aspect=11.7/8.27, kind="count", data=df2);

In [ ]:
sns.catplot(x="DUAL_INCOMES", hue="INCOME", height=8.27, aspect=11.7/8.27, kind="count", data=df2);

Молодые, люди с низким образованием и одинокие чаще всего получают небольшой доход. Что логично.

In [ ]:
sns.catplot(x="HOME_TYPE", hue="MARITAL_STATUS", height=8.27, aspect=11.7/8.27, kind="count", data=df2);

Женатые пары чаще всего живут в домах. Не совсем понятно, что это даёт бизнесу.

In [ ]:
sns.catplot(x="ETHNIC", hue="EDUCATION", height=8.27, aspect=11.7/8.27, kind="count", data=df2);

# Есть ли косяки в данных?
    1) Проверим противоречия в вопросах про замужество
    2) Посмотрим на домохозяйства, полностью состоящие из несовершеннолетних не студентов (это не обязательно косяк, но подозрительно).
    3) Посмотрим на домохозяйства, где было указано, что несовершеннолетних больше, чем всего человек.
    4) Домохозяйства, в которых больше говорят на Испанском, но чувак указал, что он не Испанец. (это не обязательно косяк, но подозрительно)
    5) Молодые люди на пенсии (это не обязательно косяк, но подозрительно)
    6) Выпускники колледжа младше 18 (это не обязательно косяк, но подозрительно)

In [ ]:
suspicious_data = df2[
    ((df2["DUAL_INCOMES"] == "Not Married") & (df2["MARITAL_STATUS"] == "Married")) | \
    (df2["PERSONS_UNDER_18"] > df["PERSONS_COUNT"]) | \
    ((df2["PERSONS_UNDER_18"] == df2["PERSONS_COUNT"]) & (df2["OCCUPATION"] != "Student, HS or College")) | \
    ((df2["ETHNIC"] != "Hispanic") & (df2["LANGUAGE"] == "Spanish")) | \
    ((df2["OCCUPATION"] == "Retired") & (df2["AGE"] < 40)) | \
    ((df2["EDUCATION"] == "5") & (df2["AGE"] < 18))
]
suspicious_data

In [ ]:
suspicious_data.shape[0] / df2.shape[0]

Итого, имеем ~3% подозрительных данных. Можно сказать, что это не очень хороший результат, т.к. мы смогли использовать не самое большое кол-во эвристик для проверки.

# Income.
Давайте глянем что будет если мы просто засунем все данные в какую-нибудь модель. Все пропуски заменим на специальное значение. За модель возьмём решающее дерево. Немного запаримся и будем обрабатывать признаки в зависимости от их типа

In [ ]:
sum(pd.isnull(df["INCOME"]))

In [ ]:
features_to_types = {
    "INCOME": "ord",
    "SEX": "bin",
    "MARITAL_STATUS": "cat",
    "AGE": "ord",
    "EDUCATION": "ord",
    "OCCUPATION": "cat",
    "LIVING_PERIOD": "ord",
    "DUAL_INCOMES": "cat",
    "PERSONS_COUNT": "ord",
    "PERSONS_UNDER_18": "ord",
    "HOUSEHOLDER_STATUS": "cat",
    "HOME_TYPE": "cat",
    "ETHNIC": "cat",
    "LANGUAGE": "cat"
}

In [ ]:
types_to_features = {}
for k, v in features_to_types.items():
    if v not in types_to_features:
        types_to_features[v] = set()
    types_to_features[v].add(k)
types_to_features

In [ ]:
df2 = df.copy()
df2 = applay_maps(df, maps_for_models)
# for column in df.columns:
#     if column in maps_for_models:
#         df2[column] = df2[column].apply(decode_column, args = (maps_for_models[column],))  

In [ ]:
df

In [ ]:
df2 = df2.fillna(-1)

In [ ]:
def encode_df(df: pd.DataFrame, features_to_types: Dict[str, str]) -> pd.DataFrame:
    df = df.copy()
    for column in df.columns:
        t = features_to_types[column]
        if t == "cat" or t == "bin":
            df = pd.concat([df.drop(column, axis=1), pd.get_dummies(df[column], prefix=column + "_is")], axis=1)
    return df

In [ ]:
df2

In [ ]:
df2 = encode_df(df2, features_to_types=features_to_types)

In [ ]:
df2

In [ ]:
y = df2["INCOME"].to_numpy()
y

In [ ]:
X = df2.drop("INCOME", axis=1).to_numpy()
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
clf = DecisionTreeClassifier(class_weight="balanced")
clf = clf.fit(X_train, y_train)

In [ ]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

In [ ]:
y_pred[100:200]

In [ ]:
y_test

# Что получили?
Даже по метрике Accuracy видно, что модель нормально так переобучилась. Попробуем задать либо максимальную глубину дерева, либо минимальное кол-во экземпляров в листах.

In [ ]:
for i in range(1, 120):
    clf = DecisionTreeClassifier(min_samples_leaf=i, class_weight="balanced")
    clf = clf.fit(X_train, y_train)
    print ("=============== " + str(i) + " ===============")
    print(clf.score(X_train, y_train))
    print(clf.score(X_test, y_test))

# Как-то не впечатляет
Хммм... Простым перебором минимального кол-ва экземпляров в листах смогли убрать переобучение и вырвали пару пунктов. Теперь попробуем поработать нормально. Начнём с заполнения пропусков.
Самый простой способ: для категориальных признаков взять наиболее частое значение, для числовых - медиану.
Конечно, можно попредсказывать пустые поля на основании остальных, но пока не будем.

In [ ]:
def simple_classifier(df: pd.DataFrame) -> None:    
    y = df["INCOME"].to_numpy()
    X = df2.drop("INCOME", axis=1).to_numpy()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    
    for i in range(1, 120):
        clf = DecisionTreeClassifier(min_samples_leaf=i, max_depth=5, class_weight="balanced")
        clf = clf.fit(X_train, y_train)
        print ("=============== " + str(i) + " ===============")
        print(clf.score(X_train, y_train))
        print(clf.score(X_test, y_test))

In [ ]:
df2 = df.copy()
for column in df2.columns:
    t = features_to_types[column]
    if t == "cat" or t == "bin":
        df2[column] = df2[column].fillna(df2[column].value_counts().index[0])
    elif t == "ord":
        df2[column] = df2[column].fillna(df2[column].median())
df2.sample(20)

In [ ]:
df2 = applay_maps(df2, maps_for_models)
df2 = encode_df(df2, features_to_types)

In [ ]:
df2

In [ ]:
simple_classifier(df2)

Попробуем добавить новых признаков

In [ ]:
df2 = df.copy()
for column in df2.columns:
    t = features_to_types[column]
    if t == "cat" or t == "bin":
        df2[column] = df2[column].fillna(df2[column].value_counts().index[0])
    elif t == "ord":
        df2[column] = df2[column].fillna(df2[column].median())

df2["PERSONS_OLDER_18"] = df2["PERSONS_COUNT"] - df2["PERSONS_UNDER_18"]
df2["MEAN_INCOME"] = (df2["PERSONS_OLDER_18"] + df2["PERSONS_UNDER_18"]*0.05)*20000
df2["SMTH1"] = (df2["PERSONS_OLDER_18"] + df2["PERSONS_UNDER_18"]*0.2)*20000*df2["AGE"]

In [ ]:
advanced_features_to_types = deepcopy(features_to_types)
advanced_features_to_types["PERSONS_OLDER_18"] = "ord"
advanced_features_to_types["MEAN_INCOME"] = "ord"
advanced_features_to_types["SMTH1"] = "ord"

In [ ]:
df2 = applay_maps(df2, maps_for_models)
df2 = encode_df(df2, advanced_features_to_types)

In [ ]:
simple_classifier(df2)

In [ ]:
pipeline = Pipeline([("classifier", XGBClassifier())])

# pipe.fit(X_train, y_train) 

params = {
 "classifier__max_depth": list(range(1, 10)),
 "classifier__n_jobs": [4],
 "classifier__n_estimators": [10, 20, 50, 100]
}
gs = GridSearchCV(estimator=pipeline, param_grid=params , scoring="f1_micro")
gs.fit(X, y)

In [ ]:
gs.best_params_, gs.best_score_

In [ ]:
clf = gs.best_estimator_
clf.predict(X_test)

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())